# Group 10 - First Year Project
### Data Science at ITU 
## Project 2 - Correlation between Covid-19's spread and weather conditions

Introduction

This notebook contains all the code developed in the Project 2 - Correlation between Covid-19's spread and weather conditions

Contact/Group: 
- Florian Micliuc(flmi@itu.dk)
- Louis Caspar Brandt (locb@itu.dk) (see disclosure statement in the report)
- Iben Mai Huse (ibhu@itu.dk)
- Katalin Literati-Dobos (klit@itu.dk)
- Ruben Jonsman (rubj@itu.dk)

# Library imports

In [ ]:
import numpy as np
import pandas as pd
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium import plugins 
from folium.plugins import HeatMap, MarkerCluster
import shapely
import json 
from shapely.geometry import Point, MultiPoint, LineString, MultiLineString, Polygon, MultiPolygon
import statsmodels.api as sm
from scipy.stats import pearsonr, spearmanr
from folium.features import DivIcon
import imageio
from matplotlib import rcParams
import calplot

# Functions

In [ ]:
def check_null_values(file, string):
    if file.isnull().values.any():
        print('There are null values in {} dataset'.format(string))
    else:
        print('There are no null values in {} dataset'.format(string))

In [ ]:
def dataset_checker_values(dataset,value):
    SA = dataset.copy()
    SA.replace(value, np.nan, inplace=True)
    missingdata_df = SA.columns[SA.isnull().any()].tolist()
    msno.matrix(SA);

In [ ]:
def weather_mean_bar_plot(column,df):
    list_month = ['February','March','April','May','June','July','August','September','October','November','December','January','February']
    fig,ax = plt.subplots(figsize = (16,4))
    df[column].plot.bar(color = 'silver',edgecolor = 'black',rot = 0)
    ax.set_title('Mean of' + ' '+ column)
    ax.set_ylabel('Value')
    ax.set_xlabel('Month')
    ax.set_xticklabels(list_month)

In [ ]:
def corona_total_bar_plot(column,df):
    list_month = ['February','March','April','May','June','July','August','September','October','November','December','January','February']
    fig,ax = plt.subplots(figsize = (16,4))
    df[column].plot.bar(color = 'silver',edgecolor = 'black',rot = 0)
    ax.set_title(column)
    ax.set_ylabel('Value')
    ax.set_xlabel('Month')
    ax.set_xticklabels(list_month)

In [ ]:
def corona_line_plot(df,x,y,title):
    ax = df.plot.line(x = x, y = y,rot = 45,figsize = (20,6),color = 'silver')
    ax.set_title(title)
    ax.set_ylabel("Covid-19 Cases")
    ax.set_xlabel("Date")
    plt.axvline(x='2020-03-15',color = '#ff6c70',ls = '--');
    plt.axvline(x='2020-03-22',color = '#f00008',ls = '--');
    plt.axvline(x='2020-04-15',color = 'blue',ls = '--');
    plt.axvline(x='2020-11-02',color = '#ff6c70',ls = '--');
    plt.axvline(x='2020-12-15',color = '#f00008',ls = '--');
    plt.axvline(x='2020-12-26',color = '#00ff68',ls = '--');

In [ ]:
def corona_bar_plot(x,y,title):
    fig = plt.figure(figsize=(20,6))
    axes = fig.add_axes([0,0,1,1])
    axes.bar(x, y,color = 'silver',edgecolor = 'black')
    axes.set_title(title)
    axes.set_ylabel("Covid-19 Cases")
    axes.set_xlabel("Date")

In [ ]:
def pearson_association(df,column,headers,significance):
    corrs = []
    pvalues = []
    for var in headers:
        corr, pvalue = pearsonr(df[column], df[var])
        print(var,'\n',round(corr,5),'   ',pvalue,pvalue<significance,'\n')

In [ ]:
def spearman_association(df,column,headers,significance):
    corrs = []
    pvalues = []
    for var in headers:
        corr, pvalue = spearmanr(df[column], df[var])
        print(var,'\n',round(corr,5),'   ',pvalue,pvalue<significance,'\n')

In [ ]:
def pearson_log(df,column,headers,significance):
    corrs = []
    pvalues = []
    for var in headers:
        corr, pvalue = pearsonr(np.log(df[column]), df[var])
        print(var,'\n',round(corr,5),'   ',pvalue,pvalue<significance,'\n')

In [ ]:
def heat_calendar(df,column,color,title):
    calplot.calplot(df[column], cmap=color,colorbar = True,figsize = (16,4),suptitle=title)

# Task 0 - Data cleaning and filtering

## Loading data

The data was loaded by creating a small pipeline to keep track of files and locations, in case they would change. This approach made it easy to change file locations if needed.

In [ ]:
PATH = {}
PATH["data_raw"] = "../data/raw/"
PATH["data_interim"] = "../data/interim/"
PATH["data_processed"] = "../data/processed/"
PATH["data_external"] = "../data/external/"

SUBPATH = {}
SUBPATH["corona"] = "corona/"
SUBPATH["meta"] = "metadata/"
SUBPATH["shape"] = "shapefiles/"
SUBPATH["weather"] = "weather/"
SUBPATH["age"] = "age/"

FILENAME = {}
FILENAME["corona"] = "de_corona.csv"
FILENAME["meta"] = "de_metadata.json"
FILENAME["shape"] = "de.geojson"
FILENAME["weather"] = "weather.csv"
FILENAME["age_pop"] = "Altersverteilung_Bevolkerung.csv"

corona = PATH["data_raw"]+SUBPATH["corona"]+FILENAME["corona"]
meta = PATH["data_raw"]+SUBPATH["meta"]+FILENAME["meta"]
shape = PATH["data_raw"]+SUBPATH["shape"]+FILENAME["shape"]
weather = PATH["data_raw"]+SUBPATH["weather"]+FILENAME["weather"]
age_pop = PATH["data_external"]+SUBPATH["age"]+FILENAME["age_pop"]

The dates is parsed in the data, so that it has a consistent form. The data is tab seperated and thus we load the data with a tab seperator. We also append the extra data that was provided to our original weather data frame.

In [ ]:
raw_data = {}

from datetime import datetime
dateparse = lambda x: datetime.strptime(x, '%Y-%m-%d')

#data
raw_data['de_corona'] = pd.read_csv(corona, sep='\t', parse_dates=['date'], date_parser=dateparse)
raw_data['weather'] = pd.read_csv(weather, sep='\t', parse_dates=['date'], date_parser=dateparse)
additional = pd.read_csv('../Data/Raw/weather/weather2.csv', sep='\t', parse_dates=['date'], date_parser=dateparse)
raw_data['weather'] = raw_data['weather'].append(additional)

#open jsonfile_metadata
with open(meta, 'r', encoding='utf-8') as f:
    country_metadata = json.loads(f.read())

#task4
raw_data['age_pop'] = pd.read_csv(age_pop, sep=';', decimal=',')

In [ ]:
HEADERS = ['de_corona','weather']
for i in HEADERS:
    print(i +"  \tDataFrame shape: " + str(raw_data[i].shape))

## Data type insight

With the data provided involved the following fields: 

In [ ]:
list(raw_data["weather"].columns), list(raw_data["de_corona"].columns)

The following fields are categorical variables:
- iso3166-2 (the codes of the regions in Germany)
- date
- region_code (the name of the region)

The following fields are numerical variables:
- RelativeHumiditySurface 
- SolarRadiation (measured j/sqm)
- Surfacepressure 
- TemperatureAboveGround (measured in Kelvin)
- Totalprecipitation
- UVIndex (1-10)
- Windspeed (m/s)
- confirmed_addition (number of confirmed Covid-19 infections)
- deceased_addition (number of deceased with a confirmed Covid-19 infections)

All the weather data is a 24-hour sum of the day, which will result in higher than expected values, these variables can be divided by 24 to get the mean of the day, though this will also be flawed data, since the data will have a daily mean. SolarRadiation, UVIndex is only relevent when the sun is actually present during the day, by getting the daily mean, the hours of the day where the sun is not active is taken into account, which results in lower than expected values. 

## Filtering the weather dataset

Our assigned country in this project is Germany and thus we filter the data so that we only have data from Germany

In [ ]:
#get Germany data
weather_data_de = raw_data["weather"][raw_data["weather"]["iso3166-2"].str.startswith("DE")]
#reset_index
weather_data_de.reset_index(drop=True, inplace=True)

#Processed data
processed_data = {}
processed_data["weather"] = weather_data_de
processed_data["weather"].to_csv("../Data/Processed/processed_weather.csv")
processed_data["de_corona"] = raw_data["de_corona"]

## Sanity check

Here we check for typical issues with raw data. We are checking for missing data, duplicate data and null values.

### Check for null values

In [ ]:
check_null_values(raw_data['de_corona'], "de_corona")
check_null_values(weather_data_de, "weather_data_de")

### Checking if our data source corrected their data by adding negative values to the previous day, to get the cummulitative sum correct

In [ ]:
print('min deaths',min(raw_data['de_corona']['deceased_addition']))
print('min cases',min(raw_data['de_corona']['confirmed_addition']))

### Checking and visualizing for weird values

In [ ]:
dataset_checker_values(weather_data_de,-999) #-999

In [ ]:
dataset_checker_values(weather_data_de,-1) #-1

In [ ]:
dataset_checker_values(raw_data['de_corona'],-999) #-999

In [ ]:
dataset_checker_values(raw_data['de_corona'],-1) #-1

There are no gaps in the plots thus -1 and -999 don't occur in both datasets.

### Checking for duplicates

In [ ]:
if True in list(raw_data["de_corona"].duplicated()):
    print("Duplicate rows")
else: 
    print("No duplicate rows")

## Data Manipulation

Mapping the region code to the iso3166_2-code

In [ ]:
corona_df = raw_data['de_corona']
ss = dict()
for i in country_metadata['country_metadata']:
    ss[i['covid_region_code']] = i['iso3166-2_code']
corona_df['region'] = corona_df['region_code'].map(ss)
corona_df

In [ ]:
celsius = pd.DataFrame(weather_data_de['TemperatureAboveGround'].subtract(273.15))
weather_data_de['Temp_Celsius'] = celsius
weather_data_de


#grouping weather conditions per day
weather_date = pd.to_datetime(weather_data_de['date'])
weather_daily = pd.DataFrame(weather_data_de.groupby(weather_date).mean())
#weather_daily

In [ ]:
#merging the corona dataset with the weather dataset for future associations. 
daily_corona_weather = corona_df.merge(weather_data_de, left_on = ["date", "region"], right_on = ["date", "iso3166-2"])
daily_corona_weather = daily_corona_weather.drop(["region_code", "region"], axis = 1)
daily_corona_weather #main

# Numerical summary

There are 16 regions

In [ ]:
list((raw_data['de_corona']['region_code']).unique())

### Daily registered cases in Germany

In [ ]:
corona_daily_sum = corona_df.groupby(corona_df['date']).sum().reset_index()
corona_daily_sum.describe()

The mean is quite high because it a heavy-tail distribution

In [ ]:
weather_daily.describe()

In [ ]:
rcParams['figure.figsize'] = 16,8

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2)
sns.boxplot(data =corona_daily_sum['confirmed_addition'], color='lightgray',ax = ax1).set_title('Daily confirmed addition registered in Germany - 2020');
sns.boxplot(data = corona_daily_sum['deceased_addition'], color='lightgray',ax = ax2).set_title('Daily deceased addition registered in Germany - 2020');

# Task 1 - Single Variable Analysis

## Corona

### Region - per capita

In [ ]:
#grouping the corona_dataset by region and adding the population

meta_df = pd.DataFrame(country_metadata['country_metadata'])
sorted_country_metadata = meta_df.sort_values(by = 'iso3166-2_name_en')
corona_by_region  = pd.DataFrame(raw_data['de_corona'].groupby(by = 'region_code').sum())
corona_by_region['iso3166-2_code'] = sorted_country_metadata['iso3166-2_code'].values
corona_by_region['population'] = sorted_country_metadata['population'].values

corona_by_region

In [ ]:
#The number of cases by region/capita
corona_by_region['confirmed/pop'] = corona_by_region['confirmed_addition']/corona_by_region['population']
corona_by_region['deceased/pop'] = corona_by_region['deceased_addition']/corona_by_region['population']

In [ ]:
corona_by_region.groupby("region_code")["deceased/pop", "confirmed/pop"].describe()

Standard deviation returns NaN because the values are too small so the standard deviation would be insignificant.

In [ ]:
rcParams['figure.figsize'] = 16,8
data_forplot= corona_by_region.copy()
data_forplot['region_code'] = data_forplot.index
fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=True)
ax = sns.barplot(x="confirmed/pop", y="region_code", data=data_forplot,ax=ax1).set_title('The number of confirmed cases by region per capita')
ax = sns.barplot(x="deceased/pop", y="region_code", data=data_forplot,ax=ax2).set_title('The number of deceased cases by region per capita');

### Per Month

In [ ]:
corona_date = pd.to_datetime(raw_data['de_corona']['date'])
corona_2020=pd.DataFrame(raw_data['de_corona'][corona_date.dt.year == 2020])
corona_2021=pd.DataFrame(raw_data['de_corona'][corona_date.dt.year == 2021])

corona_2021_month_sum = pd.DataFrame(corona_2021.groupby(corona_date.dt.month).sum())
corona_month_sum = pd.DataFrame(corona_2020.groupby(corona_date.dt.month).sum())
corona_month_mean = pd.DataFrame(corona_2020.groupby(corona_date.dt.month).mean())
#corona_month_mean
right_one = corona_month_sum.append(corona_2021_month_sum)
#right_one

In [ ]:
corona_total_bar_plot('confirmed_addition',right_one[1:])

In [ ]:
corona_total_bar_plot('deceased_addition',right_one[1:])

### Per Region

In [ ]:
corona_bar_plot(corona_by_region['iso3166-2_code'],corona_by_region['confirmed_addition'],'Confirmed Corona Cases per Region') #potential use

In [ ]:
corona_bar_plot(corona_by_region['iso3166-2_code'],corona_by_region['deceased_addition'],'Deceased Corona Cases per Region') #potential use

## Weather

In [ ]:
heat_calendar(weather_daily,'RelativeHumiditySurface','Blues_r','Relative Humidity Surface Map')

In [ ]:
heat_calendar(weather_daily,'SolarRadiation','YlOrRd','Solar Radiation Map');

In [ ]:
heat_calendar(weather_daily,'Surfacepressure','PuBu','Surface Pressure Map');

In [ ]:
heat_calendar(weather_daily,'TemperatureAboveGround','Oranges','TemperatureAboveGround Map');

In [ ]:
heat_calendar(weather_daily,'Totalprecipitation','Blues_r','Total Precipitation Map');

In [ ]:
heat_calendar(weather_daily,'UVIndex','Oranges','UVIndex Map');

UVIndex values are a daily sum so it is much higher than expected and the night is taken into consideration as well where the UVIndex is 0.

In [ ]:
heat_calendar(weather_daily,'Temp_Celsius','Oranges','Temp Celsius Map');

# Task 2 - Associations

## Daily Associations

In [ ]:
weather_headers = ['RelativeHumiditySurface', 'SolarRadiation', 'Surfacepressure', 'TemperatureAboveGround',
             'Totalprecipitation', 'UVIndex', 'WindSpeed','Temp_Celsius']
significance_threshold = 0.001 / (len(weather_headers) * 3)

We do the significance threshold with a really small numerator to make sure that it rejects the null hypothesis (it does not occur by chance). We do eight tests times three because we do three different association tests. We update the significance threshold by using the Bonferroni Correction.


### Pearson association for daily_corona_weather for original dataset

Testing for linearity in the data

In [ ]:
pearson_association(daily_corona_weather,'confirmed_addition',weather_headers,significance_threshold)

Visualizing the pearson correlation

In [ ]:
sns.regplot(x="confirmed_addition", y="Temp_Celsius", data=daily_corona_weather, color="black", ci=None,line_kws={'lw':4});

In [ ]:
sns.regplot(x="confirmed_addition", y="UVIndex", data=daily_corona_weather, color="black", ci=None, line_kws={'lw':4});

### Spearman association for daily_corona_weather for original dataset.

Testing for monotonic relationships in the data

In [ ]:
spearman_association(daily_corona_weather,'confirmed_addition',weather_headers,significance_threshold)

### Pearson asscociation for daily_corona_weather, with logtransformation of confirmed_addition

In [ ]:
pearson_log(daily_corona_weather,'confirmed_addition',weather_headers,significance_threshold)

Testing if there is a linear relationship on a non-linear scale in the data and if the data is skewed, thus a log transformation is used.


### Association tests for the deceased corona additon

In [ ]:
pearson_association(daily_corona_weather,'deceased_addition',weather_headers,significance_threshold)

In [ ]:
spearman_association(daily_corona_weather,'deceased_addition',weather_headers,significance_threshold)

In [ ]:
#can't be done because of 0 values
#pearson_log(daily_corona_weather,'deceased_addition',weather_headers,significance_threshold)

### sm.ols (Least Squares) multivariate regression


Values here may be misleading. This is because the multivariate regression, when looking at specific variables, it keeps all the other variables constant.

In [ ]:
df = sm.add_constant(daily_corona_weather)
weather_headers.extend(["const"])

In [ ]:
est = sm.OLS(df["confirmed_addition"], df[weather_headers], hasconst = True).fit()
est.summary()

Here TemperatureAboveGround is a misleading variable, since in the previous sections we analysed that having a higher temperature leads to fewer cases. Though in this multiplevariate regression it is found out that a higher temperature increases the number of cases, this contradicts all prior analysis. The reason for this is that the multivariate regression keeps all other variables constant. That is, a day with the exact same weather conditions, besides the temperature being higher, will lead to more cases. To this there could be multiple reasons for this to occur, though a major and probable reason could be that people go outside during high temperatures, which in turn will lead to people not being isolated and thus there will be a higher probability of people getting a Covid-19 infection. 

Thus all these weather variables lead to change in human behavior, which results in changes in the amount of Covid-19 cases.

In [ ]:
est = sm.OLS(df["deceased_addition"], df[weather_headers], hasconst = True).fit()
print(est.summary())

All the weather variables does not result in anything interesting to be discorvered. From the multivariate regression it is apparent that more people die, with a covid-19 infection, during higher solar radiation. It also shows that fewer people die when the total precipitation is higher. Intuitively these variables does change the human behavior, but in no way that would cause them to reduce or increase the number of deaths caused by covid-19. Also the people dying of Covid-19 is most likely in a hospital bed and is under extensive care, thus intuitively these weather variables would not lead to changes in covid-19 deaths. 

The only explanation would be that the hospital care is worsened during good weather and better during poor weather. Which also does not make much sense intuitively.

In [ ]:
#corona and weather conditions grouped by months
weather_date = pd.to_datetime(weather_data_de['date'])
weather_2020=pd.DataFrame(weather_data_de[weather_date.dt.year == 2020])
weather_month_2020 = pd.DataFrame(weather_2020.groupby(weather_date.dt.month).mean())
df_monthly_sum = corona_month_sum.merge(weather_month_2020,left_on = 'date',right_on = 'date')

In [ ]:
weather_headers_month = ['RelativeHumiditySurface', 'SolarRadiation', 'Surfacepressure', 'TemperatureAboveGround',
             'Totalprecipitation', 'UVIndex', 'WindSpeed']
significance_threshold_monthly = 0.5 / (len(weather_headers_month) * 3)
print(significance_threshold_monthly)

Here the significance threshhold is increased due to the grouping of the data into months. This means that the breakpoint of where a variable is as not being correlated by chance is higher, which means that the certainty that the variable is a correlated by chance is less precise. 

#### Corona Cases per Month

In [ ]:
pearson_association(df_monthly_sum,'confirmed_addition',weather_headers_month,significance_threshold_monthly)

In [ ]:
#a scatterplot of two variables, x and y, 
#regression line - fit a line to the data by calculating R^2 (sum of least squares)
fig, ax = plt.subplots()
ax2 = ax.twinx() #This allows the common axes (flow rate) to be shared
sns.regplot(x="confirmed_addition", y="RelativeHumiditySurface", data=df_monthly_sum, ax=ax, color="black", ci=None,line_kws={'lw':4})
sns.regplot(x="confirmed_addition", y="UVIndex", data=df_monthly_sum, color="gray", ax=ax2, ci=None, line_kws={'lw':4})

ax2.legend(handles=[a.lines[0] for a in [ax,ax2]], 
           labels=["Temp_Celsius", "UVIndex"])
plt.show()

In [ ]:
spearman_association(df_monthly_sum,'confirmed_addition',weather_headers_month,significance_threshold_monthly)

In [ ]:
pearson_log(df_monthly_sum,'confirmed_addition',weather_headers_month,significance_threshold_monthly)

From these multivariate regressions it is apparent from the pearson that there is a correlation between the monthly total of Covid-19 cases and RelativeHumiditySurface, SolarRadiation and UVIndex. These variables tend to be higher during the summer, where there was a decrease in Covid-19 cases, thus this makes sense. Though in the spearman there is also seen high correlations, though the p-value is too high for us to assume that it has not happend by chance.

#### Corona Deaths per Month

In [ ]:
pearson_association(df_monthly_sum,'deceased_addition',weather_headers_month,significance_threshold_monthly)

In [ ]:
spearman_association(df_monthly_sum,'deceased_addition',weather_headers_month,significance_threshold_monthly)

In [ ]:
pearson_log(df_monthly_sum[1:],'deceased_addition',weather_headers_month,significance_threshold_monthly) #interesting, check temperature above ground

The analysis of the corona cases/deaths per month could be interesting when it is analyzed how infectious one strain is from another. This would be quite hard to do daily because there is really hard that 1st of February 2020 would have the same weather conditions as 1st of February 2021. Therefore, there is a higher chance that the overall weather conditions in February 2020 to be the similar to February 2021.

### Dummy Variable Analysis

In [ ]:
daily_corona_weather['Lockdown'] = '0'
daily_corona_weather.loc[(daily_corona_weather["date"] >= "2020-11-02") & (daily_corona_weather["date"] <= "2021-03-07"), "Lockdown"] = '1'
daily_corona_weather['OneMonthAfter_Lockdown'] = '0'
daily_corona_weather.loc[(daily_corona_weather["date"] >= "2020-12-02") & (daily_corona_weather["date"] <= "2021-03-07"), "OneMonthAfter_Lockdown"] = '1'
daily_corona_weather

In [ ]:
mask = (corona_df['date'] > "2020-11-02") & (corona_df['date'] <= "2021-03-07")
corona_daily_sum_lockdown = corona_df[mask].groupby(corona_df[mask]['date']).sum().reset_index()
#corona_daily_sum_lockdown

In [ ]:
corona_line_plot(corona_daily_sum,'date','confirmed_addition','Corona lockdown plot')

In [ ]:
corona_line_plot(corona_daily_sum,'date','deceased_addition','Corona lockdown plot')

Data taken from https://en.wikipedia.org/wiki/COVID-19_pandemic_in_Germany - Government Reactions

- 15th March, borders to Austria, Denmark, France, Luxembourg and Switzerland were closed (light-red)
- 22nd March, curfews were imposed in six German states while other states prohibited physical contact with more than one person from outside one's household. (dark-red)
- 15th April loosening of restrictions (blue)
- 2nd November a partial lockdown was implemented, in which physical distancing rules were tightened while schools and kindergartens remained open, only temporarily halted the rise in case numbers (light-red)
- 15th December a hard lockdown was implemented (dark-red)
- 26th December 2020 - Vaccinations with the BioNTech–Pfizer COVID-19 vaccine began  (green)

In [ ]:
corona_bar_plot(corona_daily_sum_lockdown['date'],corona_daily_sum_lockdown['confirmed_addition'],'Corona Lockdown Cases')

In [ ]:
corona_bar_plot(corona_daily_sum_lockdown['date'],corona_daily_sum_lockdown['deceased_addition'],'Corona lockdown plot')

In [ ]:
x = ['RelativeHumiditySurface', 'SolarRadiation', 'Surfacepressure', 'TemperatureAboveGround',
             'Totalprecipitation', 'UVIndex', 'WindSpeed','Lockdown','const']
df1 = sm.add_constant(daily_corona_weather)
est1 = sm.OLS(np.log(df1['confirmed_addition']), df1[x].astype(float), hasconst = True).fit()
print(est1.summary())

The R-squared values changed after the addition of the lockdown variable to our analysis from 0.263 to 0.537. This means a 0.274 increase in our predictive power. It seems the lockdown has a significant positive effect on the number of cases. Intuitively this should not be true. The lockdown can not have an immediate effect. The virus has an incubation time, it would continue to spread until approximately 2 weeks into the lockdown.
To get a more accurate view of what happened after the lockdown some weeks later, a 'OneMonthAfter_lockdown' column was created. One month could be enough to assume enough people can strictly follow the restrictions to see the impact of the lockdown.

In [ ]:
x = ['RelativeHumiditySurface', 'SolarRadiation', 'Surfacepressure', 'TemperatureAboveGround',
             'Totalprecipitation', 'UVIndex', 'WindSpeed','OneMonthAfter_Lockdown','const']
df2 = sm.add_constant(daily_corona_weather)
est1 = sm.OLS(np.log(df2['confirmed_addition']), df2[x].astype(float), hasconst = True).fit()
print(est1.summary())

# Task 3 - Map visualization

## Total number of Cases of Covid-19 on a regional basis

In [ ]:
corona_by_region['confirmed/pop']

In [ ]:
n_cases = folium.Map(location = [51.5, 10.7], zoom_start = 6)

folium.Choropleth(
    geo_data = shape,
    name = "cases",
    data = corona_by_region,
    columns = ["iso3166-2_code", "confirmed_addition"],
    key_on = "properties.iso_3166_2",
    fill_color = "OrRd", 
    fill_opacity = 0.7,
    line_opacity = 0.2,
    highlight=True,
    legend_name = "Number of Cases of Covid-19").add_to(n_cases)
n_cases

## Total number of deceased with a confirmed Covid-19 infection on a regional basis


In [ ]:
dead = folium.Map(location = [51.5, 10.7], zoom_start = 6)

folium.Choropleth(
    geo_data = shape,
    name = "cases",
    data = corona_by_region,
    columns = ["iso3166-2_code", "deceased_addition"],
    key_on = "properties.iso_3166_2",
    fill_color = "Greys", 
    fill_opacity = 0.7,
    line_opacity = 0.2,
    highlight=True,
    legend_name = "Number of Deceased with an covid-19 infection").add_to(dead)
dead

## Population on a regional basis 

In [ ]:
p_cases = folium.Map(location = [51.5, 10.7], zoom_start = 6)
folium.Choropleth(
    geo_data = shape,
    name = "population",
    data = corona_by_region,
    columns = ["iso3166-2_code", "population"],
    key_on = "properties.iso_3166_2",
    fill_color = "YlGn", 
    fill_opacity = 0.7,
    line_opacity = 0.2,
    highlight=True,
    legend_name = "Population").add_to(p_cases)
p_cases

## Number of Cases pr. population of region

In [ ]:
r_cases = folium.Map(location = [51.5, 10.7], zoom_start = 6)

tiles = ['stamenwatercolor', 'cartodbpositron', 'openstreetmap', 'stamenterrain']
for tile in tiles:
    folium.TileLayer(tile).add_to(r_cases)

folium.Choropleth(
    geo_data = shape,
    name = "cases",
    data = corona_by_region,
    columns = ["iso3166-2_code", "confirmed/pop"],
    key_on = "properties.iso_3166_2",
    fill_color = "OrRd", 
    fill_opacity = 0.7,
    line_opacity = 0.2,
    legend_name = "Number of Cases pr. population of region",
    highlight=True
).add_to(r_cases)

folium.LayerControl().add_to(r_cases)

r_cases

## UVIndex on a regional basis

The UVIndex is a daily sum, this is handled by dividing by 24, because there is 24 hours in a day. This will though result in very small UVIndex values, because UVIndex during the night is taken into account.

In [ ]:
# dividing by 24 since there is 24 hours in a day, though this takes into account night where the uvindex is 
# really small/nonexsistent
# Could be argued that it should be divided by 16-18 since these are the hours where the sun is present. 
weather_by_region = pd.DataFrame(weather_data_de.groupby(by = 'iso3166-2').mean())
weather_by_region['UVIndex_scaled'] = weather_by_region['UVIndex']/24

In [ ]:
uv_index = folium.Map(location = [51.5, 10.7], zoom_start = 6)

folium.Choropleth(
    geo_data = shape,
    name = "uv-index",
    data = weather_by_region,
    columns = [weather_by_region.index, "UVIndex_scaled"],
    key_on = "properties.iso_3166_2",
    fill_color = "OrRd", 
    fill_opacity = 0.7,
    line_opacity = 0.2,
    highlight=True,
    legend_name = "UVIndex",
    smooth_factor=0).add_to(uv_index)
uv_index

## Temperature Above Ground on a regional basis

The Temperature Above Ground is a daily sum, this means that the data is flawed and is a lot higher than expected. This is handled by dividing by 24, because there is 24 hours in a day. This will though result in very small temperature values, because the temperature during the night is taken into account. Considering the flaws here this is still a more accurate measure then what the original data would have been. 

In [ ]:
weather_by_region["TemperatureAboveGround_scaled"] = weather_by_region["TemperatureAboveGround"]/24

In [ ]:
TAG = folium.Map(location = [51.5, 10.7], zoom_start = 6)

folium.Choropleth(
    geo_data = shape,
    name = "uv-index",
    data = weather_by_region,
    columns = [weather_by_region.index, "TemperatureAboveGround_scaled"],
    key_on = "properties.iso_3166_2",
    fill_color = "OrRd", 
    fill_opacity = 0.7,
    line_opacity = 0.2,
    highlight=True,
    legend_name = "average Temperature Above Ground on a daily basis").add_to(TAG)
TAG

## Multiple plots on a regional and monthy basis

In [ ]:
raw_data['de_corona']['month'] = raw_data['de_corona']['date'].dt.strftime('%B')
raw_data['de_corona']['year'] = raw_data['de_corona']['date'].dt.strftime('%Y')
population_map = {country_metadata["country_metadata"][i]["iso3166-2_code"]: country_metadata["country_metadata"][i]["population"] for i in range(len(country_metadata["country_metadata"]))}
raw_data["de_corona"]["population"] = raw_data["de_corona"]["region"].map(population_map)
raw_data["de_corona"]["confirmed_addition_ratio"] = raw_data["de_corona"]["confirmed_addition"]/raw_data["de_corona"]["population"]

In [ ]:
cases_2020_monthly = raw_data["de_corona"][(raw_data["de_corona"]["year"] == "2020")].groupby(["month", "region"]).sum()
cases_2021_monthly = raw_data["de_corona"][(raw_data["de_corona"]["year"] == "2021")].groupby(["month", "region"]).sum()

In [ ]:
confirmed_addition_ratio_maps = dict()
for i in list(raw_data["de_corona"][(raw_data["de_corona"]["year"] == "2020")]["month"].unique()):
    n_cases = folium.Map(location = [51.5, 10.7], zoom_start = 6)
    
    folium.map.Marker(
    [54.5, 2],
    icon=DivIcon(
        icon_size=(400,50),
        icon_anchor=(0,0),
        html='<div style="font-size: 20pt">{}</div>'.format(i),
        )
    ).add_to(n_cases)
    
    folium.Choropleth(
        geo_data = shape,
        name = "cases",
        data = cases_2020_monthly.loc[i].reset_index(),
        columns = ["region", "confirmed_addition_ratio"],
        key_on = "properties.iso_3166_2",
        fill_color = "OrRd", 
        fill_opacity = 0.7,
        line_opacity = 0.2,
        highlight=True,
        nan_fill_color = "purple",
        legend_name = "Number of Cases of Covid-19 pr. population of region in {} 2020".format(i)).add_to(n_cases)
    confirmed_addition_ratio_maps[i+"-2020"] = n_cases
for i in list(raw_data["de_corona"][(raw_data["de_corona"]["year"] == "2021")]["month"].unique()):
    n_cases = folium.Map(location = [51.5, 10.7], zoom_start = 6)

    folium.map.Marker(
    [54.5, 2],
    icon=DivIcon(
        icon_size=(400,50),
        icon_anchor=(0,0),
        html='<div style="font-size: 20pt">{}</div>'.format(i),
        )
    ).add_to(n_cases)
    
    folium.Choropleth(
        geo_data = shape,
        name = "cases",
        data = cases_2021_monthly.loc[i].reset_index(),
        columns = ["region", "confirmed_addition_ratio"],
        key_on = "properties.iso_3166_2",
        fill_color = "OrRd", 
        fill_opacity = 0.7,
        line_opacity = 0.2,
        highlight=True,
        nan_fill_color = "purple",
        legend_name = "Number of Cases of Covid-19 pr. population of region in {} 2021".format(i)).add_to(n_cases)
    confirmed_addition_ratio_maps[i+"-2021"] = n_cases    

In [ ]:
confirmed_addition_maps = dict()
for i in list(raw_data["de_corona"][(raw_data["de_corona"]["year"] == "2020")]["month"].unique()):
    n_cases = folium.Map(location = [51.5, 10.7], zoom_start = 6)
    
    folium.map.Marker(
    [54.5, 2],
    icon=DivIcon(
        icon_size=(400,50),
        icon_anchor=(0,0),
        html='<div style="font-size: 20pt">{}-2020</div>'.format(i),
        )
    ).add_to(n_cases)
    
    folium.Choropleth(
        geo_data = shape,
        name = "cases",
        data = cases_2020_monthly.loc[i].reset_index(),
        columns = ["region", "confirmed_addition"],
        key_on = "properties.iso_3166_2",
        fill_color = "OrRd", 
        fill_opacity = 0.7,
        line_opacity = 0.2,
        highlight=True,
        nan_fill_color = "purple",
        legend_name = "Number of Cases of Covid-19 in {} 2020".format(i)).add_to(n_cases)
    confirmed_addition_maps[i+"-2020"] = n_cases
for i in list(raw_data["de_corona"][(raw_data["de_corona"]["year"] == "2021")]["month"].unique()):
    n_cases = folium.Map(location = [51.5, 10.7], zoom_start = 6)

    folium.map.Marker(
    [54.5, 2],
    icon=DivIcon(
        icon_size=(400,50),
        icon_anchor=(0,0),
        html='<div style="font-size: 20pt">{}-2021</div>'.format(i),
        )
    ).add_to(n_cases)
    
    folium.Choropleth(
        geo_data = shape,
        name = "cases",
        data = cases_2021_monthly.loc[i].reset_index(),
        columns = ["region", "confirmed_addition"],
        key_on = "properties.iso_3166_2",
        fill_color = "OrRd", 
        fill_opacity = 0.7,
        line_opacity = 0.2,
        highlight=True,
        nan_fill_color = "purple",
        legend_name = "Number of Cases of Covid-19 in {} 2021".format(i)).add_to(n_cases)
    confirmed_addition_maps[i+"-2021"] = n_cases   
confirmed_addition = ["other_notebooks/maps/confirmed_addition/"+i+".png" for i in confirmed_addition_maps.keys()]
images = []
for filename in confirmed_addition:
    images.append(imageio.imread(filename))
imageio.mimsave('other_notebooks/map_gifs/confirmed_addition_maps.gif', images, duration=2)

In [ ]:
deceased_addition_maps = dict()
for i in list(raw_data["de_corona"][(raw_data["de_corona"]["year"] == "2020")]["month"].unique()):
    n_cases = folium.Map(location = [51.5, 10.7], zoom_start = 6)

    folium.map.Marker(
    [54.5, 2],
    icon=DivIcon(
        icon_size=(400,50),
        icon_anchor=(0,0),
        html='<div style="font-size: 20pt">{}-2020</div>'.format(i),
        )
    ).add_to(n_cases)
    
    folium.Choropleth(
        geo_data = shape,
        name = "cases",
        data = cases_2020_monthly.loc[i].reset_index(),
        columns = ["region", "deceased_addition"],
        key_on = "properties.iso_3166_2",
        fill_color = "Greys", 
        fill_opacity = 0.7,
        line_opacity = 0.2,
        highlight=True,
        nan_fill_color = "purple",
        legend_name = "Number of deceased with a Covid-19 infection in {} 2020".format(i)).add_to(n_cases)
    deceased_addition_maps[i+"-2020"] = n_cases
for i in list(raw_data["de_corona"][(raw_data["de_corona"]["year"] == "2021")]["month"].unique()):
    n_cases = folium.Map(location = [51.5, 10.7], zoom_start = 6)

    folium.map.Marker(
    [54.5, 2],
    icon=DivIcon(
        icon_size=(400,50),
        icon_anchor=(0,0),
        html='<div style="font-size: 20pt">{}-2021</div>'.format(i),
        )
    ).add_to(n_cases)
    
    folium.Choropleth(
        geo_data = shape,
        name = "cases",
        data = cases_2021_monthly.loc[i].reset_index(),
        columns = ["region", "deceased_addition"],
        key_on = "properties.iso_3166_2",
        fill_color = "Greys", 
        fill_opacity = 0.7,
        line_opacity = 0.2,
        highlight=True,
        nan_fill_color = "purple",
        legend_name = "Number of deceased with a Covid-19 infection in {} 2021".format(i)).add_to(n_cases)
    deceased_addition_maps[i+"-2021"] = n_cases   
deceased_addition = ["other_notebooks/maps/deceased_addition/"+i+".png" for i in confirmed_addition_maps.keys()]
images = []
for filename in deceased_addition:
    images.append(imageio.imread(filename))
imageio.mimsave('other_notebooks/map_gifs/deceased_addition_maps.gif', images, duration=2)

### Viewing the maps

All the monthly maps can be ascessed by doing 
```python
deceased_addition_maps["January-2020"]
confirmed_addition_maps["January-2020"]
confirmed_addition_ratio_maps["January-2020"]
```
Missing data, or there was no recorded data in that region, the color of the region will be purple

Month and year can be changed

In [ ]:
confirmed_addition_maps["January-2020"]

![SegmentLocal](other_notebooks/map_gifs/confirmed_addition_maps.gif)

In [ ]:
deceased_addition_maps["January-2020"]

![SegmentLocal](other_notebooks/map_gifs/deceased_addition_maps.gif)

In [ ]:
confirmed_addition_ratio_maps["January-2020"]

##  Task 4: Open question
Use the data to formulate, motivate, answer, and discuss another research question of your choice. For example, compare your country to another one in the dataset; or investigate additional variables that you could
find elsewhere (e.g. total population, population density, population demographics such as age distribution,
number of elders, etc); or identify if there are temporal patterns (is one variable more important in some
months? Are weekends different from weekdays? Etc), ..

##### We loaded an external data set from the website of 'Robert Koch Institut' (RKI) to explore the role of age in potentially driving differences in infection spread.
The data set is about  COVID-19 cases by age group / 100,000 inhabitants in Germany from the 17th of March 2020 to the 2nd of March 2021.  
https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Daten/Altersverteilung.html

#### The weekly mean of total confirmed cases per 100.000 inhabitants across age groups
We added the mean of the weekly cases by age to the table to get a quick insight into the age distribution.

In [ ]:
raw_data['age_pop']['mean'] = raw_data['age_pop'].mean(axis=1)
raw_data['age_pop']

In [ ]:
raw_data['age_pop']['mean'].describe()

#### Visualize distribution of weekly mean of total confirmed cases per 100.000 per age

In [ ]:
rcParams['figure.figsize'] = 15,8
sns.set_style("whitegrid");

In [ ]:
ax = sns.barplot(x="mean", y="Altersgruppe", data=raw_data['age_pop']).set_title('The weekly mean of total confirmed cases per 100.000 people across age groups')

The weekly registered infections among people above 90 is significantly higher than in other age groups. 

The reason does not seem to be the population size of each group. (RKI: 40-59-year-olds make up the largest age group in Germany, at 23.6 million people, the next-largest age group was 65 years and older, at 18.09 million.)

#### Temporal distribution of cases across age groups

In [ ]:
df_T = raw_data['age_pop'].T
modified = df_T.reset_index()
new_header = modified.iloc[0] #grab the first row for the header
modified = modified[1:] #take the data less the header row
modified.columns = new_header #set the header row as the df header
modified.rename(columns={"Altersgruppe":"date"}, inplace=True)
modified = modified.drop([53])
modified['date'] = modified['date'].astype(str) + '_2' #2 means Tuesday
modified['date'] =  pd.to_datetime(modified['date'], format='%Y_%W_%w')

for column in modified:
    modified.iloc[:, 1:21] = modified.iloc[:, 1:21].astype(float)

# merge age groups
modified['0-19'] = modified.loc[:,['0 - 4','5 - 9', '10 - 14', '15 - 19']].sum(axis=1)
modified['20-39'] = modified.loc[:,['20 - 24','25 - 29', '30 - 34', '35 - 39']].sum(axis=1)
modified['40-59'] = modified.loc[:,['40 - 44', '45 - 49', '50 - 54', '55 - 59']].sum(axis=1)
modified['60-79'] = modified.loc[:,['60 - 64','65 - 69', '70 - 74', '75 - 79']].sum(axis=1)
modified['80+'] = modified.loc[:,['80 - 84', '85 - 89', '90+']].sum(axis=1)
#modified

The pandemic is exhibiting seasonal patterns in its spread. It is clear that there is a peak in the number of cases in the winter period of 2020-2021 and the number of cases are low during the summer of 2020. 
Next, the distribution of these specific periods will be plotted by ages.

In [ ]:
x = modified.drop(modified.columns.difference(['date', '0-19', '20-39', '40-59', '60-79', '80+']), axis=1)
melt_modified = x.melt('date', var_name='age_gaps',  value_name='cases')

g = sns.lineplot(x="date", y="cases", hue='age_gaps', data=melt_modified).set_title('All weekly registered cases in Germany by age groups')

In [ ]:
mask_winter = (melt_modified['date'].dt.month > 10)
winter = melt_modified[mask_winter]

In [ ]:
mask_summer = ((melt_modified['date'].dt.month < 9) & (melt_modified['date'].dt.month > 5))
summer = melt_modified[mask_summer]

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(16,4))
sns.lineplot(x="date", y="cases", hue='age_gaps',ax=ax2, data=winter).set_title('2020-2021: WINTER: All weekly registered cases in Germany by age groups');
plt.xticks(rotation=30)
sns.lineplot(x="date", y="cases", hue='age_gaps', ax=ax1, data=summer).set_title('2020 SUMMER: All weekly registered cases in Germany by age groups');

However, during the summer, when the overall number of cases are low, the proportion of the infected among young adults are higher, in the age group 20-39 the highest.

By investigating the possible connections between the weather variables and the number of cases by age groups, the focus will be on people from 20 to 39 years and above 80.

In [ ]:
#merge corona_age & weather WEEKLY for plots
#corona age: 2020.03.12 - 2021.03.02.  - weather: 2020-02-13 - 2020.11.14.

#prepare weather data to merge without regions
df_w = weather_data_de.copy()
#sum some weather values:
df_w_sum = df_w.groupby('date')['RelativeHumiditySurface', 'SolarRadiation', 'Surfacepressure', 'Totalprecipitation'].sum()
#take the mean some weather values:
df_w_mean = df_w.groupby('date')['TemperatureAboveGround', 'UVIndex', 'WindSpeed'].mean()
df_w_noregions = df_w_sum.merge(df_w_mean, left_on = ["date"], right_on = ["date"]) 
#date index to column data:
df_w_noregions.reset_index(level=0, inplace=True)

#merge (modified is the weekly corona data by age)
weekly_coronaAge_weather = modified.merge(df_w_noregions, left_on = ["date"], right_on = ["date"])
weekly_coronaAge_weather.drop_duplicates(subset='date');

celsius = pd.DataFrame(weekly_coronaAge_weather['TemperatureAboveGround'].subtract(273.15))
weekly_coronaAge_weather['Temp_Celsius'] = celsius

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=True)
sns.regplot(x="80+", y="Temp_Celsius", data=weekly_coronaAge_weather, ax=ax1, color="black", ci=None)
sns.regplot(x="20-39", y="Temp_Celsius", data=weekly_coronaAge_weather, ax=ax2, color="black", ci=None);

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=True)
sns.regplot(x="80+", y="UVIndex", data=weekly_coronaAge_weather, ax=ax1, color="black", ci=None)
sns.regplot(x="20-39", y="UVIndex", data=weekly_coronaAge_weather, ax=ax2, color="black", ci=None);

### Multivariate linear regression using statsmodel for daily_coronaAge_weather 80+

Is there any difference in how each weather variable can influence the number of cases in these two specific age groups? 

In [ ]:
#DAILY DATA
#There is data for 52 weeks: create daily data - divide the case numbers by 7
modified['date'] = pd.to_datetime(modified['date'])
daily_age = modified.set_index('date').resample('d').mean().ffill().div(7).reset_index()


#changes in weather data: change values: sum and mean
#df_w = de_weather.copy()
#df_w_sum = df_w.groupby('date')['RelativeHumiditySurface', 'SolarRadiation', 'Surfacepressure', 'Totalprecipitation'].sum()
#df_w_mean = df_w.groupby('date')['TemperatureAboveGround', 'UVIndex', 'WindSpeed'].mean()
#no need for regions
df_w_noregions = df_w_sum.merge(df_w_mean, left_on = ["date"], right_on = ["date"]) 
daily_coronaAge_weather = daily_age.merge(df_w_noregions,  left_on = ["date"], right_on = ["date"])
daily_coronaAge_weather = daily_coronaAge_weather.drop_duplicates(subset='date')

celsius = pd.DataFrame(daily_coronaAge_weather['TemperatureAboveGround'].subtract(273.15))
daily_coronaAge_weather['Temp_Celsius'] = celsius
#daily_age
#daily_coronaAge_weather

In [ ]:
daily_coronaAge_weather = sm.add_constant(daily_coronaAge_weather)

est = sm.OLS(np.log(daily_coronaAge_weather['80+']), daily_coronaAge_weather[weather_headers], hasconst = True).fit()
print(est.summary())

### Multivariate linear regression using statsmodel for daily_coronaAge_weather 20-39

In [ ]:
est = sm.OLS(np.log(daily_coronaAge_weather['20-39']), daily_coronaAge_weather[weather_headers], hasconst = True).fit()
print(est.summary())

R-squared 0.760 and 0.882 (corr.coeff. squared) It means this model can explain the 76% and 82.2% of the variation in our data. 
Comparing the two multivariate linear regression model results shows that from the 7 weather variables, 6 has similar impact on the numbers of cases for these two age groups, the only exeption is the temperature above ground. At the age group 20-39 it is positive(0.0087),  at people above 80 is negative(-0.1188).

While analyzing how the COVID-19 infection spreads across different age groups, the most straightforward patterns in the temporal data distribution were that COVID-19 cases are highest among young adults during summer and significantly higher among people above 80 during winter. 
With a linear regression model, we could support this fact.
Intuitively, when the numbers of covid cases are low and the weather is fine, young adults, who are not the most endangered age group, are more likely to take risks against social distancing. 
The possible explanation for higher infection rates across people above 80 can be that they are biologically the most susceptible population for covid-19.
